<a href="https://colab.research.google.com/github/Balghanimi/AutoGPT/blob/master/first_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import zipfile
import os

# Path to the uploaded file
zip_path = next(iter(uploaded))  # gets the uploaded filename

# Extract the ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/PDSMC_OAELM')

# Verify extraction
!ls /content/PDSMC_OAELM


**Reasoning**:
I need to inspect the content of `quad_helpers.py` to understand why `params` might be `None` in the `simulate_oa_elm` function. This requires accessing the file system in the Colab environment, which cannot be done directly with a Python `code_block`. I will use a shell command to display the file content.



quad_helpers.py

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

# === ELM Class ===
class OnlineAdaptiveELM:
    def __init__(self, input_size=6, hidden_size=40, gamma=0.05):
        rng = np.random.default_rng(42)
        self.W = rng.uniform(-1, 1, (hidden_size, input_size))
        self.b = rng.uniform(0, 1, (hidden_size, 1))
        self.beta = np.zeros((hidden_size, 1))
        self.gamma = gamma
        self.mu = np.zeros((input_size, 1))
        self.sigv = np.ones((input_size, 1))

    def activation(self, X):
        return 1 / (1 + np.exp(-(self.W @ X + self.b)))

    def normalize(self, X_raw, i):
        if i > 0:
            self.mu = 0.95 * self.mu + 0.05 * X_raw
            self.sigv = 0.95 * self.sigv + 0.05 * (X_raw - self.mu)**2
        else:
            self.mu = X_raw
            self.sigv = np.ones_like(X_raw)
        return np.clip((X_raw - self.mu) / np.sqrt(self.sigv + 1e-6), -3, 3)

    def predict(self, X):
        H = self.activation(X)
        return float(self.beta.T @ H)

    def update(self, X, target):
        H = self.activation(X)
        d_hat = float(self.beta.T @ H)
        self.beta += self.gamma * H * (target - d_hat)
        return d_hat

# === Simulation Functions ===
def simulate_oa_elm(ref_val, init_val, d_func, lam, k, gamma, n_hidden, axis, params):
    t = np.arange(0, params['t_end'], params['dt'])
    x, dx = init_val, 0.0
    ref = ref_val * np.ones_like(t)
    log = {'state': [], 'd_true': [], 'd_hat': [], 's': [], 'u_total': []}
    elm = OnlineAdaptiveELM(6, n_hidden, gamma)

    for i, ti in enumerate(t):
        e = ref[i] - x
        de = -dx
        s = de + lam * e
        u_eq = lam * de + lam**2 * e
        u_sw = k * np.tanh(s)
        d = d_func(ti)
        d_hat = elm.update(elm.normalize(np.array([[e], [de], [s], [0.0], [ref[i]], [x]]), i), d)
        u = u_eq + u_sw - d_hat
        ddx = u + d

        dx += ddx * params['dt']
        x += dx * params['dt']

        log['state'].append(x)
        log['d_true'].append(d)
        log['d_hat'].append(d_hat)
        log['s'].append(s)
        log['u_total'].append(u)

    return t, ref, log

def simulate_pd_smc(ref_val, init_val, d_func, lam, k, axis, params):
    t = np.arange(0, params['t_end'], params['dt'])
    x, dx = init_val, 0.0
    ref = ref_val * np.ones_like(t)
    log = {'state': [], 'd_true': [], 's': [], 'd_hat': [], 'u_total': []}

    for ti in t:
        e = ref[0] - x
        de = -dx
        s = de + lam * e
        u_eq = lam * de + lam**2 * e
        u_sw = k * np.tanh(s)
        u = u_eq + u_sw
        d = d_func(ti)
        ddx = u + d

        dx += ddx * params['dt']
        x += dx * params['dt']

        log['state'].append(x)
        log['d_true'].append(d)
        log['s'].append(s)
        log['d_hat'].append(0.0)
        log['u_total'].append(u)

    return t, ref, log


Main SMC_ELM code

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import scipy.io as sio
from IPython.display import display, clear_output
from pathlib import Path

# === System Parameters ===
params = {
    'dt': 0.01,
    't_end': 10,
    'm': 0.468,
    'g': 9.81,
    'Ixx': 4.856e-3,
    'Iyy': 4.856e-3,
    'Izz': 8.801e-3
}

# === Disturbance Function ===
def d_real(t, mode='sine', A=1.0, f=0.5):
    if mode == 'sine':
        return A * np.sin(2 * np.pi * f * t)
    elif mode == 'step':
        return A if t >= 2 else 0
    elif mode == 'impulse':
        return A * np.exp(-((t - 2)**2) / (2 * (0.1**2)))
    return 0.0

# === GUI Setup ===
lam = widgets.FloatSlider(value=5.5, min=0.5, max=30.0, step=0.1, description='λ')
k = widgets.FloatSlider(value=50, min=10, max=150, step=1, description='k')
gamma = widgets.FloatSlider(value=0.05, min=0.001, max=0.1, step=0.001, description='γ')
hidden = widgets.IntSlider(value=40, min=10, max=100, step=5, description='Hidden')
disturb = widgets.Dropdown(options=['sine', 'step', 'impulse'], description='Disturb')
axis_select = widgets.Dropdown(options=['z', 'phi', 'theta', 'psi'], value='z', description='Axis')
run_button = widgets.Button(description='Run SMC vs SMC_OA_ELM')
output = widgets.Output()

ui = widgets.VBox([
    widgets.HBox([lam, k]),
    widgets.HBox([gamma, hidden]),
    widgets.HBox([disturb, axis_select]),
    run_button,
    output
])
display(ui)

# === Callback ===
def on_run_clicked(b):
    with output:
        clear_output()
        axis = axis_select.value
        d_func = lambda t: d_real(t, disturb.value)

        ref_vals = {'z': 5.0, 'phi': -0.3, 'theta': 0.0, 'psi': 0.3}
        init_vals = {'z': 0.0, 'phi': 0.0, 'theta': -0.9, 'psi': 0.0}

        ref_val = ref_vals[axis]
        init_val = init_vals[axis]

        t, ref, log_oa = simulate_oa_elm(ref_val, init_val, d_func, lam.value, k.value, gamma.value, hidden.value, axis, params)
        _, _, log_smc = simulate_pd_smc(ref_val, init_val, d_func, lam.value, k.value, axis, params)

        fig, axs = plt.subplots(3, 1, figsize=(12, 10), sharex=True)
        axs[0].plot(t, ref, 'k--', label='Reference')
        axs[0].plot(t, log_oa['state'], 'b', label='SMC_OA_ELM')
        axs[0].plot(t, log_smc['state'], 'r--', label='SMC')
        axs[0].set_title('Tracking'); axs[0].legend(); axs[0].grid()

        axs[1].plot(t, log_oa['d_true'], 'k', label='True d')
        axs[1].plot(t, log_oa['d_hat'], 'b', label='SMC_OA_ELM Estimate')
        axs[1].set_title('Disturbance Estimation'); axs[1].legend(); axs[1].grid()

        axs[2].plot(t, log_oa['s'], 'b', label='SMC_OA_ELM s')
        axs[2].plot(t, log_smc['s'], 'r--', label='SMC s')
        axs[2].set_title('Sliding Surface'); axs[2].legend(); axs[2].grid()

        plt.tight_layout()
        plt.show()

        sio.savemat(f"/content/compare_smc_oaelm_{axis}.mat", {
            't': t,
            'ref': ref,
            'smc_oaelm_state': np.array(log_oa['state']),
            'smc_oaelm_d_hat': np.array(log_oa['d_hat']),
            'smc_oaelm_s': np.array(log_oa['s']),
            'smc_state': np.array(log_smc['state']),
            'smc_s': np.array(log_smc['s']),
            'd_true': np.array(log_oa['d_true'])
        })
        print(f"✅ Results saved to compare_smc_oaelm_{axis}.mat")

run_button.on_click(on_run_clicked)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output
import sys


# Initialize latest_result
latest_result = None

# === System Parameters ===
params = {
    'dt': 0.01,
    't_end': 10,
    'm': 0.468,
    'g': 9.81,
    'Ixx': 4.856e-3,
    'Iyy': 4.856e-3,
    'Izz': 8.801e-3
}

# === Disturbance Function ===
def d_real(t, mode='sine', A=1.0, f=0.5):
    if mode == 'sine':
        return A * np.sin(2 * np.pi * f * t)
    elif mode == 'step':
        return A if t >= 2 else 0
    elif mode == 'impulse':
        return A * np.exp(-((t - 2)**2) / (2 * (0.1**2)))
    elif mode == 'none':
        return 0.0
    return 0.0

# === GUI Widgets ===

lambda_slider = widgets.FloatSlider(value=5.5, min=0.1, max=20, step=0.1, description='lambda')
k_slider = widgets.FloatSlider(value=50, min=10, max=150, step=1, description='k')
gamma_slider = widgets.FloatSlider(value=0.05, min=0.001, max=0.1, step=0.001, description='gamma')
hidden_slider = widgets.IntSlider(value=40, min=10, max=100, step=5, description='Hidden')
disturb_dropdown = widgets.Dropdown(options=['none', 'sine', 'step', 'impulse'], value='sine', description='Disturbance')
axis_dropdown = widgets.Dropdown(options=['z', 'phi', 'theta', 'psi'], value='z', description='Axis')
run_button = widgets.Button(description='Run Comparison')
export_pdf_button = widgets.Button(description='📄 Export Plot as PDF')
export_mat_button = widgets.Button(description='📂 Export Data as .mat')
export_all_button = widgets.Button(description='🗂 Export All Axes Results')
output = widgets.Output()


ui = widgets.VBox([
    widgets.HBox([lambda_slider, k_slider]),
    widgets.HBox([gamma_slider, hidden_slider]),
    widgets.HBox([disturb_dropdown, axis_dropdown]),
    run_button,
    widgets.HBox([export_pdf_button, export_mat_button,export_all_button]),
    output
])

display(ui)

# === Output Directory ===
output_dir = Path("/content/PDSMC_OAELM/PDSMC_OAELM/outputs")
output_dir.mkdir(parents=True, exist_ok=True)

latest_result = None

def export_all_results(b):
    with output:
        clear_output()
        lam = lambda_slider.value
        k = k_slider.value
        gamma = gamma_slider.value
        hidden = hidden_slider.value

        ref_vals = {'z': 5.0, 'phi': -0.3, 'theta': 0.0, 'psi': 0.3}
        init_vals = {'z': 0.0, 'phi': 0.0, 'theta': -0.9, 'psi': 0.0}
        disturbances = ['none', 'sine', 'step', 'impulse']  # ← All types

        for disturb in disturbances:
            for axis in ['z', 'phi', 'theta', 'psi']:
                print(f"\nExporting for {axis.upper()} axis with '{disturb}' disturbance...")
                d_func = lambda t: d_real(t, mode=disturb)
                ref_val = ref_vals[axis]
                init_val = init_vals[axis]
                t, ref, log_oa = simulate_oa_elm(ref_val, init_val, d_func, lam, k, gamma, hidden, axis, params)
                _, _, log_smc = simulate_pd_smc(ref_val, init_val, d_func, lam, k, axis, params)

                # === Save PDF ===
                fig, axs = plt.subplots(2, 2, figsize=(14, 8))
                axs[0, 0].plot(t, ref, 'k--', label='Ref')
                axs[0, 0].plot(t, log_oa['state'], color='orange', label='OA-ELM')
                axs[0, 0].plot(t, log_smc['state'], color='purple', linestyle='--', label='SMC')
                axs[0, 0].set_title('Tracking'); axs[0, 0].legend(); axs[0, 0].grid()

                axs[0, 1].plot(t, ref - np.array(log_oa['state']), color='orange')
                axs[0, 1].plot(t, ref - np.array(log_smc['state']), color='purple', linestyle='--')
                axs[0, 1].set_title('Tracking Error'); axs[0, 1].grid()

                axs[1, 0].plot(t, log_oa.get('u_total', []), color='orange')
                axs[1, 0].plot(t, log_smc.get('u_total', []), color='purple', linestyle='--')
                axs[1, 0].set_title('Control Effort'); axs[1, 0].grid()

                axs[1, 1].plot(t, log_oa['s'], color='orange')
                axs[1, 1].plot(t, log_smc['s'], color='purple', linestyle='--')
                axs[1, 1].set_title('Sliding Surface'); axs[1, 1].grid()

                plt.tight_layout()
                pdf_path = output_dir / f"compare_smc_oaelm_{axis}_{disturb}.pdf"
                plt.savefig(pdf_path, bbox_inches='tight', dpi=300)
                plt.close(fig)
                print(f"📄 Saved: {pdf_path.name}")

                # === Save .mat ===
                mat_path = output_dir / f"compare_smc_oaelm_{axis}_{disturb}.mat"
                sio.savemat(mat_path, {
                    't': t,
                    'ref': ref,
                    'smc_oaelm_state': np.array(log_oa['state']),
                    'smc_oaelm_d_hat': np.array(log_oa['d_hat']),
                    'smc_oaelm_s': np.array(log_oa['s']),
                    'smc_oaelm_u': np.array(log_oa.get('u_total', [])),
                    'smc_state': np.array(log_smc['state']),
                    'smc_s': np.array(log_smc['s']),
                    'smc_u': np.array(log_smc.get('u_total', [])),
                    'd_true': np.array(log_oa['d_true'])
                })
                print(f"💾 Saved: {mat_path.name}")
        print("✅ All axes and disturbances exported.")


# === Bind Buttons ===
run_button.on_click(on_run_clicked)
export_all_button.on_click(export_all_results)


code for saving result

## Save results

### Subtask:
Save the simulation time, true disturbance, and estimated disturbance for each run to a separate file (e.g., a `.mat` file) for later analysis or plotting.


Estimation code

**Reasoning**:
Import necessary libraries for saving data and define the output directory.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import ipywidgets as widgets
from IPython.display import display, clear_output
from pathlib import Path
import sys

# Add the directory containing quad_helpers.py to sys.path
# Assuming quad_helpers.py is in '/content/PDSMC_OAELM/PDSMC_OAELM/helpers'
helpers_path = '/content/PDSMC_OAELM/PDSMC_OAELM/helpers'
if helpers_path not in sys.path:
    sys.path.append(helpers_path)

# === System Parameters ===
params = {
    'dt': 0.01,
    't_end': 10,
    'm': 0.468,
    'g': 9.81,
    'Ixx': 4.856e-3,
    'Iyy': 4.856e-3,
    'Izz': 8.801e-3
}

# === Disturbance Function ===
def d_real(t, mode='sine', A=1.0, f=0.5):
    if mode == 'sine':
        return A * np.sin(2 * np.pi * f * t)
    elif mode == 'step':
        return A if t >= 2 else 0
    elif mode == 'impulse':
        return A * np.exp(-((t - 2) ** 2) / (2 * (0.1 ** 2)))
    return 0.0

# The import statement was moved to a new cell.
# The rest of the code in this cell depends on the successful import.

# === Set up parameters ===
axes_to_simulate = ['z', 'phi', 'theta', 'psi']
disturb_types = ['sine', 'step', 'impulse']

ref_vals = {'z': 5.0, 'phi': -0.3, 'theta': -0.9, 'psi': 0.3}
init_vals = {'z': 0.0, 'phi': 0.0, 'theta': 0.0, 'psi': 0.0}

lam_val = 5.5
k_val = 50
gamma_val = 0.05
hidden_val = 40

# === Output Directory ===
output_dir = Path("/content/PDSMC_OAELM/outputs")
output_dir.mkdir(parents=True, exist_ok=True)

# === Batch Simulation Loop ===
print(f"Starting batch simulations for axes: {axes_to_simulate} and disturbance types: {disturb_types}")

simulation_results = {}

for axis in axes_to_simulate:
    simulation_results[axis] = {}
    for disturb_type in disturb_types:
        print(f"\nRunning simulation for axis: {axis} with '{disturb_type}' disturbance...")
        d_func = lambda t: d_real(t, mode=disturb_type)
        ref_val = ref_vals[axis]
        init_val = init_vals[axis]

        # Run simulation
        t, ref, log_oa = simulate_oa_elm(ref_val, init_val, d_func, lam_val, k_val, gamma_val, hidden_val, axis, params)

        # Store results
        simulation_results[axis][disturb_type] = {
            't': t,
            'ref': ref,
            'log_oa': log_oa
        }
        print("Simulation finished.")

print("\nBatch simulations complete.")

# === Save Results (.mat) ===
print("Saving simulation results to .mat files...")

for axis in axes_to_simulate:
    for disturb_type in disturb_types:
        result = simulation_results[axis][disturb_type]
        t = result['t']
        log_oa = result['log_oa']

        # Construct filename
        mat_filename = output_dir / f"disturbance_estimation_{axis}_{disturb_type}.mat"

        # Save data to .mat file
        sio.savemat(mat_filename, {
            't': t,
            'd_true': np.array(log_oa['d_true']),
            'd_hat': np.array(log_oa['d_hat'])
        })
        print(f"💾 Data saved to: {mat_filename}")

print("\nSaving complete.")

# === Optional: Generate Plots (.pdf) ===
print("Generating and saving plots as PDF...")

for axis in axes_to_simulate:
    for disturb_type in disturb_types:
        result = simulation_results[axis][disturb_type]
        t = result['t']
        log_oa = result['log_oa']

        d_true = np.array(log_oa['d_true'])
        d_hat = np.array(log_oa['d_hat'])
        error_d = d_true - d_hat

        # Create plot
        fig, axs = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

        # Plot true vs. estimated disturbance
        axs[0].plot(t, d_true, 'k', label='True d')
        axs[0].plot(t, d_hat, 'g', label='OA-ELM Estimate')
        axs[0].set_title(f'{axis.upper()} Axis Disturbance Estimation ({disturb_type.capitalize()} Disturbance)')
        axs[0].set_ylabel('Disturbance')
        axs[0].legend(); axs[0].grid()

        # Plot estimation error
        axs[1].plot(t, error_d, 'r', label='Estimation Error')
        axs[1].set_title('Estimation Error')
        axs[1].set_xlabel('Time (s)')
        axs[1].set_ylabel('Error')
        axs[1].legend(); axs[1].grid()

        plt.tight_layout()

        # Construct filename and save plot as PDF
        plot_filename = output_dir / f"disturbance_estimation_plot_{axis}_{disturb_type}.pdf"
        fig.savefig(plot_filename, bbox_inches='tight', dpi=300, format='pdf') # Save as PDF
        plt.close(fig) # Close the figure

        print(f"📄 Plot saved to: {plot_filename}")

print("\nPlot generation and saving complete.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output


# === System Parameters ===
params = {
    'dt': 0.01,
    't_end': 10,
    'm': 0.468,
    'g': 9.81,
    'Ixx': 4.856e-3,
    'Iyy': 4.856e-3,
    'Izz': 8.801e-3
}

# === Disturbance Function ===
def d_real(t, mode='sine', A=1.0, f=0.5):
    if mode == 'sine':
        return A * np.sin(2 * np.pi * f * t)
    elif mode == 'step':
        return A if t >= 2 else 0
    elif mode == 'impulse':
        return A * np.exp(-((t - 2)**2) / (2 * (0.1**2)))
    return 0.0

# === Widgets ===
axis_dropdown = widgets.Dropdown(options=['z', 'phi', 'theta', 'psi'], value='z', description='Axis:')
disturb_dropdown = widgets.Dropdown(options=['sine', 'step', 'impulse'], value='sine', description='Disturb:')
gamma_slider = widgets.FloatSlider(value=0.05, min=0.001, max=0.1, step=0.001, description='γ')
hidden_slider = widgets.IntSlider(value=40, min=10, max=100, step=5, description='Hidden')
run_button = widgets.Button(description='Run & Save Results')
output = widgets.Output()

ui = widgets.VBox([
    widgets.HBox([axis_dropdown, disturb_dropdown]),
    widgets.HBox([gamma_slider, hidden_slider]),
    run_button,
    output
])
display(ui)

# === Callback ===
def on_run_clicked(b):
    with output:
        clear_output()
        axis = axis_dropdown.value
        disturb = disturb_dropdown.value
        gamma = gamma_slider.value
        hidden = hidden_slider.value
        d_func = lambda t: d_real(t, mode=disturb)

        ref_vals = {'z': 5.0, 'phi': -0.3, 'theta': 0.0, 'psi': 0.3}
        init_vals = {'z': 0.0, 'phi': 0.0, 'theta': -0.9, 'psi': 0.0}
        ref_val = ref_vals[axis]
        init_val = init_vals[axis]

        print(f"🛫 Simulating for Axis: {axis}, Disturbance: {disturb}, γ: {gamma}, Hidden: {hidden}")
        t, ref, log_oa = simulate_oa_elm(ref_val, init_val, d_func, 5.5, 50, gamma, hidden, axis, params)

        # === Plot ===
        fig, axs = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
        axs[0].plot(t, log_oa['d_true'], 'k', label='True d')
        axs[0].plot(t, log_oa['d_hat'], 'g', label='OA-ELM Estimate')
        axs[0].set_title('Disturbance Estimation')
        axs[0].legend(); axs[0].grid()

        error_d = np.array(log_oa['d_true']) - np.array(log_oa['d_hat'])
        axs[1].plot(t, error_d, 'r', label='Estimation Error')
        axs[1].set_title('Estimation Error')
        axs[1].legend(); axs[1].grid()

        plt.tight_layout()
        plt.show()

        # === Save ===
        output_dir = Path("/content/PDSMC_OAELM/outputs")
        output_dir.mkdir(parents=True, exist_ok=True)

        mat_path = output_dir / f"disturbance_estimation_{axis}_{disturb}.mat"
        sio.savemat(mat_path, {
            't': t,
            'd_true': np.array(log_oa['d_true']),
            'd_hat': np.array(log_oa['d_hat']),
            'error_d': error_d
        })
        print(f"💾 .mat saved to: {mat_path}")

        pdf_path = output_dir / f"disturbance_estimation_{axis}_{disturb}.pdf"
        fig.savefig(pdf_path, bbox_inches='tight', dpi=300)
        print(f"📄 PDF saved to: {pdf_path}")

run_button.on_click(on_run_clicked)


In [ ]:
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# Define output directory
output_dir = Path("/content/PDSMC_OAELM/outputs")

# Load two result files to compare
mat1 = sio.loadmat(output_dir / "disturbance_estimation_z_sine.mat")
mat2 = sio.loadmat(output_dir / "disturbance_estimation_phi_sine.mat")

# Extract data
t1, d_true1, d_hat1 = mat1['t'].flatten(), mat1['d_true'].flatten(), mat1['d_hat'].flatten()
error_d1 = d_true1 - d_hat1

t2, d_true2, d_hat2 = mat2['t'].flatten(), mat2['d_true'].flatten(), mat2['d_hat'].flatten()
error_d2 = d_true2 - d_hat2


# Create side-by-side comparison figure
fig, axs = plt.subplots(2, 2, figsize=(12, 8))

# Z-Axis Results
axs[0, 0].plot(t1, d_true1, 'k', label='True d')
axs[0, 0].plot(t1, d_hat1, 'g', label='OA-ELM Estimate')
axs[0, 0].set_title('Z-Axis Disturbance Estimation (Sine)'); axs[0, 0].legend(); axs[0, 0].grid()
axs[0, 0].set_xlabel('Time (s)')
axs[0, 0].set_ylabel('Disturbance')


axs[0, 1].plot(t1, error_d1, 'r', label='Estimation Error')
axs[0, 1].set_title('Z-Axis Estimation Error (Sine)'); axs[0, 1].legend(); axs[0, 1].grid()
axs[0, 1].set_xlabel('Time (s)')
axs[0, 1].set_ylabel('Error')


# Phi-Axis Results
axs[1, 0].plot(t2, d_true2, 'k', label='True d')
axs[1, 0].plot(t2, d_hat2, 'g', label='OA-ELM Estimate')
axs[1, 0].set_title('Phi-Axis Disturbance Estimation (Sine)'); axs[1, 0].legend(); axs[1, 0].grid()
axs[1, 0].set_xlabel('Time (s)')
axs[1, 0].set_ylabel('Disturbance')


axs[1, 1].plot(t2, error_d2, 'r', label='Estimation Error')
axs[1, 1].set_title('Phi-Axis Estimation Error (Sine)'); axs[1, 1].legend(); axs[1, 1].grid()
axs[1, 1].set_xlabel('Time (s)')
axs[1, 1].set_ylabel('Error')


plt.tight_layout()
plt.suptitle("Comparison of Disturbance Estimation (Z and Phi Axes)", fontsize=16, y=1.02)
plt.subplots_adjust(top=0.92)

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from pathlib import Path

# Load two .mat results (assuming paths are already known)
path_z = "/content/PDSMC_OAELM/outputs/disturbance_estimation_z_sine.mat"
path_phi = "/content/PDSMC_OAELM/outputs/disturbance_estimation_phi_sine.mat"

data_z = sio.loadmat(path_z)
data_phi = sio.loadmat(path_phi)

# Set up plot styles
colors = {
    "ref": "black",
    "oa": "darkorange",
    "smc": "purple"
}

labels = {
    "z": "Altitude (z)",
    "phi": "Roll (ϕ)"
}

fig, axs = plt.subplots(2, 2, figsize=(16, 8), sharex=True)
axs = axs.flatten()

# --- z axis ---
t_z = data_z['t'].flatten()
ref_z = data_z['ref'].flatten()
oa_z = data_z['smc_oaelm_state'].flatten()
smc_z = data_z['smc_state'].flatten()
s_z_oa = data_z['smc_oaelm_s'].flatten()
s_z_smc = data_z['smc_s'].flatten()

axs[0].plot(t_z, ref_z, linestyle='--', color=colors['ref'], label='Reference')
axs[0].plot(t_z, oa_z, color=colors['oa'], label='SMC-OA-ELM')
axs[0].plot(t_z, smc_z, linestyle='-.', color=colors['smc'], label='SMC')
axs[0].set_title("z: Tracking")
axs[0].legend()
axs[0].grid()

axs[1].plot(t_z, ref_z - oa_z, color=colors['oa'], label='SMC-OA-ELM Error')
axs[1].plot(t_z, ref_z - smc_z, color=colors['smc'], linestyle='--', label='SMC Error')
axs[1].set_title("z: Error")
axs[1].legend()
axs[1].grid()

# --- phi axis ---
t_phi = data_phi['t'].flatten()
ref_phi = data_phi['ref'].flatten()
oa_phi = data_phi['smc_oaelm_state'].flatten()
smc_phi = data_phi['smc_state'].flatten()
s_phi_oa = data_phi['smc_oaelm_s'].flatten()
s_phi_smc = data_phi['smc_s'].flatten()

axs[2].plot(t_phi, ref_phi, linestyle='--', color=colors['ref'], label='Reference')
axs[2].plot(t_phi, oa_phi, color=colors['oa'], label='SMC-OA-ELM')
axs[2].plot(t_phi, smc_phi, linestyle='-.', color=colors['smc'], label='SMC')
axs[2].set_title("ϕ: Tracking")
axs[2].legend()
axs[2].grid()

axs[3].plot(t_phi, ref_phi - oa_phi, color=colors['oa'], label='SMC-OA-ELM Error')
axs[3].plot(t_phi, ref_phi - smc_phi, color=colors['smc'], linestyle='--', label='SMC Error')
axs[3].set_title("ϕ: Error")
axs[3].legend()
axs[3].grid()

plt.tight_layout()
plt.show()